In [ ]:
import pandas as pd
import numpy as np
import datetime
import plotly.express as px
import plotly.graph_objs as go

In [ ]:
from google.colab import files

# Upload files manually
uploaded = files.upload()

# Access the uploaded files
for file_name in uploaded.keys():
    print(f"Uploaded file: {file_name}")


Saving Customers.csv to Customers.csv
Uploaded file: Customers.csv


In [ ]:
from google.colab import files

# Upload files manually
uploaded = files.upload()

# Access the uploaded files
for file_name in uploaded.keys():
    print(f"Uploaded file: {file_name}")


Saving Products.csv to Products.csv
Uploaded file: Products.csv


In [ ]:
from google.colab import files

# Upload files manually
uploaded = files.upload()

# Access the uploaded files
for file_name in uploaded.keys():
    print(f"Uploaded file: {file_name}")


Saving Transactions.csv to Transactions.csv
Uploaded file: Transactions.csv


Task 1: Exploratory Data Analysis (EDA) and Business Insights

In [ ]:
# Load datasets
customers = pd.read_csv("Customers.csv")
products = pd.read_csv("Products.csv")
transactions = pd.read_csv("Transactions.csv")

In [ ]:
# Explore datasets
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   CustomerID    200 non-null    object        
 1   CustomerName  200 non-null    object        
 2   Region        200 non-null    object        
 3   SignupDate    200 non-null    datetime64[ns]
dtypes: datetime64[ns](1), object(3)
memory usage: 6.4+ KB


In [ ]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ProductID    100 non-null    object 
 1   ProductName  100 non-null    object 
 2   Category     100 non-null    object 
 3   Price        100 non-null    float64
dtypes: float64(1), object(3)
memory usage: 3.3+ KB


In [ ]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   TransactionID    1000 non-null   object        
 1   CustomerID       1000 non-null   object        
 2   ProductID        1000 non-null   object        
 3   TransactionDate  1000 non-null   datetime64[ns]
 4   Quantity         1000 non-null   int64         
 5   TotalValue       1000 non-null   float64       
 6   Price            1000 non-null   float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(3)
memory usage: 54.8+ KB


In [ ]:
print(transactions.shape)
print(products.shape)
print(customers.shape)

(1000, 7)
(100, 4)
(200, 4)


In [17]:
# Data Preprocessing
# Convert date columns to datetime
customers['SignupDate'] = pd.to_datetime(customers['SignupDate'])
transactions['TransactionDate'] = pd.to_datetime(transactions['TransactionDate'])

In [18]:
# Merge datasets
merged_data = transactions.merge(customers, on='CustomerID')
merged_data = merged_data.merge(products, on='ProductID')

In [19]:
# Basic statistics
summary_stats = merged_data.describe()
summary_stats

,TransactionDate,Quantity,TotalValue,Price_x,SignupDate,Price_y
count,1000,1000.000000,1000.000000,1000.00000,1000,1000.00000
mean,2024-06-23 15:33:02.768999936,2.537000,689.995560,272.55407,2023-07-09 02:49:55.199999744,272.55407
min,2023-12-30 15:29:12,1.000000,16.080000,16.08000,2022-01-22 00:00:00,16.08000
25%,2024-03-25 22:05:34.500000,2.000000,295.295000,147.95000,2022-09-17 12:00:00,147.95000
50%,2024-06-26 17:21:52.500000,3.000000,588.880000,299.93000,2023-07-23 00:00:00,299.93000
75%,2024-09-19 14:19:57,4.000000,1011.660000,404.40000,2024-04-12 00:00:00,404.40000
max,2024-12-28 11:00:00,4.000000,1991.040000,497.76000,2024-12-28 00:00:00,497.76000
std,NaN,1.117981,493.144478,140.73639,NaN,140.73639


In [20]:
missing_values = merged_data.isnull().sum()
missing_values

,0
TransactionID,0
CustomerID,0
ProductID,0
TransactionDate,0
Quantity,0
TotalValue,0
Price_x,0
CustomerName,0
Region,0
SignupDate,0


In [21]:
# Distribution of sales by region
sales_by_region = merged_data.groupby("Region")["TotalValue"].sum()
sales_by_region

,TotalValue
Region,
Asia,152074.97
Europe,166254.63
North America,152313.40
South America,219352.56


In [22]:
# Distribution of sales by product category
sales_by_category = merged_data.groupby("Category")["TotalValue"].sum()
sales_by_category

,TotalValue
Category,
Books,192147.47
Clothing,166170.66
Electronics,180783.50
Home Decor,150893.93


In [24]:
# 1. Customer Acquisition Analysis
def customer_acquisition_analysis(customers):
    # Signup trend over time
    customers['SignupYear'] = customers['SignupDate'].dt.year
    customers['SignupMonth'] = customers['SignupDate'].dt.month

    signup_trend = customers.groupby(['SignupYear', 'SignupMonth']).size().reset_index(name='NewCustomers')
    signup_trend['YearMonth'] = signup_trend['SignupYear'].astype(str) + '-' + signup_trend['SignupMonth'].astype(str)

    # Using Plotly for visualization
    fig = px.line(signup_trend, x='YearMonth', y='NewCustomers',
                  title='Customer Signup Trend',
                  labels={'YearMonth': 'Year-Month', 'NewCustomers': 'Number of New Customers'})
    fig.write_html("customer_signup_trend.html")

    return signup_trend

In [25]:
# 2. Regional Sales Distribution
def regional_sales_distribution(merged_data):
    region_sales = merged_data.groupby('Region')['TotalValue'].agg(['sum', 'mean', 'count']).reset_index()
    region_sales.columns = ['Region', 'TotalSales', 'AvgSaleValue', 'NumberOfTransactions']
    region_sales = region_sales.sort_values('TotalSales', ascending=False)

    # Using Plotly for bar chart
    fig = px.bar(region_sales, x='Region', y='TotalSales',
                 title='Total Sales by Region',
                 labels={'TotalSales': 'Total Sales (USD)', 'Region': 'Region'})
    fig.write_html("regional_sales.html")

    return region_sales

In [26]:
# 3. Product Category Performance
def product_category_performance(merged_data):
    category_sales = merged_data.groupby('Category').agg({
        'TotalValue': ['sum', 'mean'],
        'Quantity': ['sum', 'mean'],
        'TransactionID': 'count'
    }).reset_index()
    category_sales.columns = ['Category', 'TotalSales', 'AvgSaleValue', 'TotalQuantitySold', 'AvgQuantityPerTransaction', 'NumberOfTransactions']
    category_sales = category_sales.sort_values('TotalSales', ascending=False)

    # Using Plotly for pie chart
    fig = px.pie(category_sales, values='TotalSales', names='Category',
                 title='Sales Distribution by Product Category')
    fig.write_html("category_sales.html")

    return category_sales

In [27]:
# 4. Customer Retention and Repeat Purchase Analysis
def customer_retention_analysis(merged_data):
    # Count transactions per customer
    customer_transaction_counts = merged_data.groupby('CustomerID')['TransactionID'].count().reset_index()
    customer_transaction_counts.columns = ['CustomerID', 'TransactionCount']

    # Using Plotly for histogram
    fig = px.histogram(customer_transaction_counts, x='TransactionCount',
                       title='Distribution of Transaction Counts per Customer',
                       labels={'TransactionCount': 'Number of Transactions', 'count': 'Number of Customers'})
    fig.write_html("customer_transaction_distribution.html")

    # Repeat customer rate
    total_customers = merged_data['CustomerID'].nunique()
    repeat_customers = customer_transaction_counts[customer_transaction_counts['TransactionCount'] > 1]['CustomerID'].count()
    repeat_customer_rate = repeat_customers / total_customers * 100

    return {
        'TotalCustomers': total_customers,
        'RepeatCustomers': repeat_customers,
        'RepeatCustomerRate': repeat_customer_rate
    }

In [28]:
# 5. Seasonal Sales Trends
def seasonal_sales_trends(merged_data):
    merged_data['SaleMonth'] = merged_data['TransactionDate'].dt.month
    monthly_sales = merged_data.groupby('SaleMonth')['TotalValue'].sum().reset_index()

    # Using Plotly for line chart
    fig = px.line(monthly_sales, x='SaleMonth', y='TotalValue',
                  title='Monthly Sales Trend',
                  labels={'SaleMonth': 'Month', 'TotalValue': 'Total Sales (USD)'})
    fig.write_html("monthly_sales_trend.html")

    return monthly_sales

In [29]:
# Run analyses
signup_trend = customer_acquisition_analysis(customers)
region_sales = regional_sales_distribution(merged_data)
category_sales = product_category_performance(merged_data)
retention_analysis = customer_retention_analysis(merged_data)
monthly_sales = seasonal_sales_trends(merged_data)

In [30]:
# Print key insights
print("Business Insights Summary:")
print("\n1. Customer Acquisition:")
print(signup_trend.nlargest(3, 'NewCustomers'))

print("\n2. Regional Sales Performance:")
print(region_sales)

print("\n3. Product Category Performance:")
print(category_sales)

print("\n4. Customer Retention:")
print(f"Repeat Customer Rate: {retention_analysis['RepeatCustomerRate']:.2f}%")

print("\n5. Seasonal Sales:")
print(monthly_sales)

Business Insights Summary:

1. Customer Acquisition:
    SignupYear  SignupMonth  NewCustomers YearMonth
32        2024            9            11    2024-9
34        2024           11            11   2024-11
27        2024            4            10    2024-4

2. Regional Sales Performance:
          Region  TotalSales  AvgSaleValue  NumberOfTransactions
3  South America   219352.56    721.554474                   304
1         Europe   166254.63    710.489872                   234
2  North America   152313.40    624.235246                   244
0           Asia   152074.97    697.591606                   218

3. Product Category Performance:
      Category  TotalSales  AvgSaleValue  TotalQuantitySold  \
0        Books   192147.47    711.657296                681   
2  Electronics   180783.50    711.746063                627   
1     Clothing   166170.66    728.818684                590   
3   Home Decor   150893.93    608.443266                639   

   AvgQuantityPerTransaction  Nu